In [1]:
# conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [2]:
import random

import torch
device = torch.device("cpu")

from transformers import AutoTokenizer, AutoModelForCausalLM

import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [3]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [5]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=5.0, 
        temperature=1.1,
        num_beams=5,
        length_penalty=0.1,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [6]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
history = []
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Нейросеть не знает, что говорит, и может сказать всякое —\n если что, не обижайтесь. Распространяя получившиеся\n тексты, помните об ответственности.')

def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    
    history.append(txt)
    result = respond_to_dialog(history[-10:])
    history.append(result)
    
    update.message.reply_text(result)
    
    with open("log.txt", "a") as file:
        print(str(update.message.chat_id) + ": " + txt, file=file)
        print("bot: " + result, file=file)

In [ ]:
updater = Updater("put your telegram bot private token here", use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

C:\Users\Nikolay\anaconda3\lib\site-packages\transformers\generation_utils.py:2158: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
